In [1]:
import sys 
import os 
import torch
from transformers import BertTokenizerFast 

project_root = os.path.abspath("..")  # se il notebook è in /report
sys.path.append(project_root)

from src.data.loader import DataLoader

print("DataLoader module imported successfully.")

c:\Users\cola0\anaconda3\envs\nlp-project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DataLoader module imported successfully.


In [2]:
MAX_LENGTH = 64
BATCH_SIZE = 4
NUM_SAMPLES = 50  

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

loader = DataLoader(
    tokenizer, 
    max_length=MAX_LENGTH, 
    batch_size=BATCH_SIZE
)

print(f"Tokenizer e DataLoader inizializzati. Cerca i primi {NUM_SAMPLES} articoli.")

Tokenizer e DataLoader inizializzati. Cerca i primi 50 articoli.


In [3]:
print("🔄 Test Data Loading pipeline...")

# 3. Ottieni DataLoader
try:
    train_loader = loader.get_dataloader(split="train", num_samples=NUM_SAMPLES)
except Exception as e:
    print(f"❌ Errore nella creazione del DataLoader: {e}")
    raise e

print("⏳ Fetching first batch...")

# Proviamo a prendere il primo batch
try:
    first_batch = next(iter(train_loader))
    print("\n✅ Batch caricato con successo!")
except Exception as e:
    print(f"\n❌ Errore nel caricamento dati: {e}")
    raise e

🔄 Test Data Loading pipeline...
 Loading Wikipedia (train) in streaming mode...


c:\Users\cola0\anaconda3\envs\nlp-project\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cola0\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Filtering the dataset...
Tokenizing the dataset...
⏳ Fetching first batch...

✅ Batch caricato con successo!


In [4]:
# Verifica delle Shape
expected_shape = torch.Size([BATCH_SIZE, MAX_LENGTH])

print("--- Risultati del Batch ---")
print(f"   Keys: {first_batch.keys()}")
print(f"   Input IDs Shape: {first_batch['input_ids'].shape} (Atteso: {expected_shape})") 
print(f"   Labels Shape: {first_batch['labels'].shape} (Atteso: {expected_shape})")       

# Analisi delle Labels
# -100 indica i token NON mascherati/NON da predire (standard MLM)
# Qualsiasi valore >= 0 indica il token originale (quello da predire)
masked_count = (first_batch['labels'] != -100).sum().item()
print(f"   Totale token mascherati nel batch: {masked_count}")

print("\n--- Visualizzazione Labels del primo esempio (solo i primi 10 token) ---")
print(f"   Labels: {first_batch['labels'][0][:10].tolist()}")

if first_batch['input_ids'].shape == expected_shape and masked_count > 0:
    print("\n✅ Test dei dati superato: le dimensioni sono corrette e il mascheramento MLM è attivo.")
else:
    print("\n❌ Test fallito.")

--- Risultati del Batch ---
   Keys: KeysView({'input_ids': tensor([[  101, 12411,  5558,  2053, 11748,  4801,  4360,  1017,  1024,  1026,
          4895,  2243,   103, 11906,  1006,  2887,  1024,   103,  1806,  1671,
         30222, 30218, 30259, 30227, 30255, 30258, 30219,   103,  1010,  5507,
           103, 11748,  4801,  4360,   103,  1996, 11686,  1017,  1007,  1010,
          4141,  3615,  2000,  2004, 11748,  4801,  4360, 11906,  3523,  2648,
          2900,  1010,  2003,  1037,  8608,  2535,   103,  1011,  1030,   103,
          2678,   103,  2764,   102],
        [  101,  1996,  2208,  2211,  2458,  1999,  2230,  1010,  4755,  2058,
          1037,  2312,   103,  1997,  1996,  2147,  2589,  2006, 11748,  4801,
          4360, 11906,  2462,  1012,  2096,  2009,  6025,  1996,  3115,  2838,
          1997,  1996,  2186, 10022,  2009,  2036,  9601,  3674, 24081,  1010,
          2107,   103,  2437,  1996,  2208,   103,  1026,  4895,  2243,  1028,
          2005,  2186, 24159,   1